#### Import libraries

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sys, os

# Random Baseball Game Generator

This notebook randomly generates baseball games based on the baserunning speed and slugging average of the players in each lineup.

There are seven outcomes per at-bat:
  1. Single
  2. Double
  3. Triple
  4. Home run
  5. Fly out (including sacrifice flies)
  6. Ground out (including double plays)
  7. Strike out

These outcomes are determined using a random outcome from a beta distribution where `beta = 3` and `mean = player's slugging pct`. 20% of singles are converted into home runs (to adjust for the distribution underpredicting HRs).

Speed is defined as the probability a runner advances to third on a single, scores from second on a single, scores from first on a double, scores on a sacrifice fly, or advances to third on a ground out. These probabilities are considered the same for simplicity.

Possible improvements:
  1. Include other kinds of outs (baserunning errors)
  2. Include triples (they are raw, but converted into singles and doubles when they do occur)
  3. Include base stealing (based on speed)
  4. Toggle on/off the print statements to allow for simulation analysis

In [156]:
def generate_at_bat_result(slug):
    result = round(np.random.beta(3*slug/(4-slug), 3)*4)
    # Turn 20% of singles into home runs
    if result == 1:
        if np.random.binomial(1, 0.2):
            result = 4
    return result

In [157]:
def generate_inning(current_batter, runs, batter_slug, batter_speed, inning, away_runs=10000):
    outs = 0
    bases = [None]*3
    while outs < 3:
        print(bases)
        result = generate_at_bat_result(batter_slug[current_batter - 1])
        if result == 1:
            print(f'Batter {current_batter} hits a single')
            if bases[2]:
                print(f'Batter {bases[2]} scores from third')
                runs += 1
                bases[2] = None
            if bases[1]:
                if np.random.binomial(1, batter_speed[bases[1] - 1]):
                    print(f'Batter {bases[1]} scores from second')
                    runs += 1
                else:
                    print(f'Batter {bases[1]} advances to third')
                    bases[2] = bases[1]
                bases[1] = None
            if bases[0]:
                if not bases[2] and np.random.binomial(1, batter_speed[bases[0] - 1]):
                    print(f'Batter {bases[0]} advances to third')
                    bases[2] = bases[0]
                else:
                    print(f'Batter {bases[0]} advances to second')
                    bases[1] = bases[0]
            bases[0] = current_batter
        elif result == 2:
            print(f'Batter {current_batter} hits a double')
            if bases[2]:
                print(f'Batter {bases[2]} scores from third')
                runs += 1
                bases[2] = None
            if bases[1]:
                print(f'Batter {bases[1]} scores from second')
                runs += 1
            if bases[0]:
                if np.random.binomial(1, batter_speed[bases[0] - 1]):
                    print(f'Batter {bases[0]} scores from first')
                    runs += 1
                else:
                    print(f'Batter {bases[0]} advances to third')
                    bases[2] = bases[0]
            bases[1] = current_batter
            bases[0] = None
        elif result == 3:
            print(f'Batter {current_batter} hits a triple')
            if bases[2]:
                print(f'Batter {bases[2]} scores from third')
                runs += 1
            if bases[1]:
                print(f'Batter {bases[1]} scores from second')
                runs += 1
            if bases[0]:
                print(f'Batter {bases[0]} scores from first')
                runs += 1
            bases[2] = current_batter
            bases[1] = bases[0] = None
        elif result == 4:
            print(f'Batter {current_batter} hits a home run!')
            runs += 1
            if bases[2]:
                print(f'Batter {bases[2]} scores from third')
                runs += 1
            if bases[1]:
                print(f'Batter {bases[1]} scores from second')
                runs += 1
            if bases[0]:
                print(f'Batter {bases[0]} scores from first')
                runs += 1
            bases = [None]*3
        else:
            # 0: strikeout, 1: fly out, 2: ground out
            out_type = np.random.choice(3, p=[.287, .341, .372])
            if out_type == 1:
                print(f'Batter {current_batter} flies out')
                outs += 1
                # Sacrifice fly
                if outs < 3 and bases[2]:
                    if np.random.binomial(1, batter_speed[bases[2] - 1]):
                        print(f'Batter {bases[2]} scores from third')
                        runs += 1
                        bases[2] = None
            elif out_type == 2:
                print(f'Batter {current_batter} grounds out')
                outs += 1
                # Runner scores from third
                if outs < 3 and bases[2]:
                    if np.random.binomial(1, batter_speed[bases[2] - 1]):
                        print(f'Batter {bases[2]} scores from third')
                        runs += 1
                        bases[2] = None
                # Runner advances from second
                if outs < 3 and bases[1]:
                    if np.random.binomial(1, batter_speed[bases[1] - 1]):
                        print(f'Batter {bases[1]} advances to third')
                        bases[2] = bases[1]
                        bases[1] = None
                # Double play
                if outs < 2 and bases[0]:
                    if np.random.binomial(1, 0.12):
                        print(f'Batter {current_batter} hits into a double play')
                        outs += 1
                        bases[0] = None
                    else:
                        print(f'Batter {bases[0]} advances to second')
                        bases[1] = bases[0]
                        bases[0] = None
            else:
                print(f'Batter {current_batter} strikes out')
                outs += 1
        if current_batter < 9:
            current_batter += 1
        else:
            current_batter = 1
        if inning >= 9 and runs > away_runs:
            print('\nWALK-OFF WIN FOR THE HOME TEAM')
            break
    return current_batter, runs

In [158]:
def generate_game(away_speed, away_slug, home_speed, home_slug):
    home_runs = 0
    away_runs = 0
    home_batter = 1
    away_batter = 1
    for i in range(9):
        away_batter, away_runs = generate_inning(away_batter, away_runs, away_slug, away_speed, i + 1)
        print(f'\nTop of inning {i+1} is over, score is {away_runs}-{home_runs}\n')
        if i + 1 != 9 or home_runs <= away_runs:
            home_batter, home_runs = generate_inning(
                home_batter, home_runs, home_slug, home_speed, i + 1, away_runs)
            print(f'\nBottom of inning {i+1} is over, score is {away_runs}-{home_runs}\n')
    if home_runs == away_runs:
        print('Score is tied, going to extra innings\n')
        while home_runs == away_runs:
            i += 1
            away_batter, away_runs = generate_inning(away_batter, away_runs, away_slug, away_speed, i + 1)
            print(f'\nTop of inning {i+1} is over, score is {away_runs}-{home_runs}\n')
            home_batter, home_runs = generate_inning(
                home_batter, home_runs, home_slug, home_speed, i + 1, away_runs)
            print(f'\nBottom of inning {i+1} is over, score is {away_runs}-{home_runs}\n')
    print(f'GAME OVER, final score is {away_runs}-{home_runs}\n')
    return away_runs, home_runs, i + 1

### Input player attributes

In [159]:
away_speed = [0.5]*9
# Yankees 2020 stats
away_slug = [.610, .590, .414, .365, .490, .368, .511, .554, .392]

home_speed = [0.5]*9
# Indians 2020 stats
home_slug = [.415, .408, .607, .450, .350, .383, .318, .286, .216]

### Generate a random game

In [164]:
generate_game(away_speed, away_slug, home_speed, home_slug)

[None, None, None]
Batter 1 strikes out
[None, None, None]
Batter 2 grounds out
[None, None, None]
Batter 3 flies out

Top of inning 1 is over, score is 0-0

[None, None, None]
Batter 1 hits a single
[1, None, None]
Batter 2 flies out
[1, None, None]
Batter 3 strikes out
[1, None, None]
Batter 4 strikes out

Bottom of inning 1 is over, score is 0-0

[None, None, None]
Batter 4 hits a single
[4, None, None]
Batter 5 hits a home run!
Batter 4 scores from first
[None, None, None]
Batter 6 grounds out
[None, None, None]
Batter 7 strikes out
[None, None, None]
Batter 8 strikes out

Top of inning 2 is over, score is 2-0

[None, None, None]
Batter 5 grounds out
[None, None, None]
Batter 6 flies out
[None, None, None]
Batter 7 flies out

Bottom of inning 2 is over, score is 2-0

[None, None, None]
Batter 9 strikes out
[None, None, None]
Batter 1 flies out
[None, None, None]
Batter 2 hits a single
[2, None, None]
Batter 3 flies out

Top of inning 3 is over, score is 2-0

[None, None, None]
Batt

(6, 5, 11)

### Generate 10,000 random games and get win probability for the home team

In [165]:
%%capture
games = []
extra_innings = []
home_runs = []
away_runs = []
for _ in range(10000):
    game = generate_game(away_speed, away_slug, home_speed, home_slug)
    if game[0] > game[1]:
        games.append(0)
    else:
        games.append(1)
    away_runs.append(game[0])
    home_runs.append(game[1])
    extra_innings.append(game[2])

In [166]:
np.mean(games)

0.2925


In [167]:
pd.Series(home_runs).describe()

count    10000.000000
mean         3.906200
std          2.499045
min          0.000000
25%          2.000000
50%          4.000000
75%          5.000000
max         21.000000
dtype: float64

In [168]:
pd.Series(away_runs).describe()

count    10000.00000
mean         6.31770
std          3.40525
min          0.00000
25%          4.00000
50%          6.00000
75%          8.00000
max         25.00000
dtype: float64